# AI算法工程师
## 毕业项目开题报告

沈捷 2019年3月28日

### 背景

2013年，kaggle举办了一项趣味性赛事“猫狗大战”，即猫狗图像分类。许多学者及行业人士参与其中，供献了大量的解决方案，机器学习领域的局势由之改变。<cite data-cite="undefined"></cite>现今图像分类在许多行业已展开深入研究，如医学上热门的癌组织与良性组织的鉴别，甚至有学者尝试利用深度学习方法鉴别出男女大脑MRI图像的差异<cite data-cite="2636952/IJ2W7TIB"></cite>，都是类似的问题。但医学领域常由于患者隐私保护政策，难以获得大量的训练数据，所以为了更好地学习、研究新的方法，也有学者会利用猫狗大战数据进行探索尝试和验证<cite data-cite="2636952/YVAJHILJ"></cite>，同理可以延伸至许多应用领域。

### 问题描述

区分图片是猫还是狗的照片，显然是一个二分类问题，通过输入图像特征，获得一个概率，通过概率来判断属于哪一个类。

### 数据输入

该数据集已切分为训练集和测试集。训练集包含25000张图片，猫和狗图片比例为1:1，可以从文件名（*标签.编号.jpg*）中获得标签；测试集有12500张图片，文件名只有编号。

随机抽取样本，简单探查这些图片：

![samples](samples.jpg)

可见图片长或宽多在300~500像素之间，大小不一。背景也各种各样，也许会造成一定程度的干扰，不过目标主体基本清晰居中。只不过有的主体占图片的比例较小，有的图片上有两个或以上的目标，也可能会影响模型训练，且尚不知是否有分类错误的训练样本。

对于错误分类样本的检测，决定采用预训练的ResNet50模型进行初始预测，根据ImageNet1000的标签，其中有118个狗的品种和7个猫的品种，可以做为参考，找到那些预测与标签不符的图片，再做人工确认。如下为ResNet50模型中找到的部分错误标签的图：

![miss_labeled](ex_files_resnet50_top60_sample.jpg)

可见，这些图中有的目标主体太小，有些有遮挡，有些模糊，有些曝光度太高以至特征消失，甚至一张图片上同时有猫和狗或两者都不是，这都是对模型训练的干扰，这部分图片将剔除。

其余训练集图片会经过随机旋转、缩放、裁剪等预处理，使模型能有更好的泛化能力。这些图像将统一为$224 \times 224$大小，并分解为RGB三个颜色通道的色值，作为模型的输入。

### 解决方案

处理好输入数据后，将采用一个预训练的神经网络进行迁移学习，预计将采用ResNet50模型。该模型要求输入为(224, 224, 3)的tensor，所以图片也会随之调整，同时神经网络的输入节点也应为$224 \times 224$大小，输出节点数量为1，采用sigmoid激活函数将图像对某一类的线性预测值映射到$(0,1)$之间，即为概率（狗=1，猫=0）。sigmoid函数公式：

$$S(x) = \frac{1}{1+e^{-x}}$$

最后将模型在测试集上进行预测，提交到kaggle上进行比较判断。模型的训练目标即是达到kaggle的前10%。

训练模型的过程中，将调整epoch、隐藏层节点、学习速率等超参数，以尽量高准确率，降低对数损失（log loss）。

### 基准模型

此处将采用vgg16模型做为基准模型，进行迁移学习。vgg16是用ImageNet的图像进行预训练的深度卷积网络模型，具有16个权重层<cite data-cite="2636952/UPQRGRJI"></cite>，可以迁移到未见过的图像进行训练，大大提高训练模型的效率。

### 评估指标

模型将采用对数损失函数（log loss）进行评价。<cite data-cite="2636952/55RLJIS4"></cite>对数损失函数需要输入每个分类的预测概率与标签，对错误的分类进行惩罚，从而对准确率（Accuracy）进行量化<cite data-cite="2636952/K3687SF8"></cite>。损失越少，准确率越高。对数损失函数公式<cite data-cite="2636952/55RLJIS4"></cite>：

$$LogLoss = -\frac{1}{n}\sum_{i=1}^{n}[y_{i}log(\hat{y_i})+(1-y_i)log(1-\hat{y_i})]$$

其中：
* n是样本数量
* $\hat{y_i}$是图像$i$为狗的预测概率值
* $y_i$是图像$i$的标签，$y_i=1$是狗，$y_i=0$是猫
* $log()$是以自然数$e$为底的对数函数

此外，预测结果将上传到_Kaggle_进行排名对比，目标是要达到_Kaggle_排名的前10%，即LogLoss达到0.05603。

### 项目设计

综上，本项目是一个图像的二分类问题。对于训练集图像，首先要进行随机翻转、缩放、裁剪等预处理，并统一成$224 \times 224$大小，再转化为RGB三个通道的Tensor，与预训练模型的标准化参数进行匹配，作为模型的输入。将采用Pytorch的vgg16为预训练模型框架进行迁移学习，用Sigmoid函数输出图像的预测概率（狗=1，猫=0）。在训练过程中，将调整epoch、隐藏层数、隐藏节点数、学习速率等超参数，尽量提高准确率，降低对数损失。在测试集上进行预测之后，提交到_kaggle_上进行评价对比。_kaggle_则采用对数损失函数(log loss）进行模型准确率的评价。本模型的目标是达到_Kaggle_排名的前10%。

### 参考文献

<div class="cite2c-biblio"></div>